Load data.

Here, I have imdb Preprocessed data.  

In [3]:
import pandas as pd

data = pd.read_csv("../../Downloads/Data for ML/imdb.txt")
print(data.head())

   class                                         clean_text
0      0  slow moving aimless movie distressed drifting ...
1      0  not sure lost flat characters audience nearly ...
2      0  attempting artiness black white clever camera ...
3      0                        little music anything speak
4      1  best scene movie gerardo trying find song keep...


Convert the text data into Vector format. 

--> Using Bag of words.
--> Always split data first before preprocessing. But here we already got preprocessed data so don't worry about that.  

In [26]:
from sklearn.model_selection import train_test_split
X = data['clean_text'].values
Y = data['class'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

print(f'Total number of data points: {len(X)}')
print(f"Number of data points in training set: {len(x_train)}")
print(f"Length of cross-validation set: {len(x_cv)}")
print(f"Number of data points in test set: {len(x_test)}")

Total number of data points: 746
Number of data points in training set: 476
Length of cross-validation set: 120
Number of data points in test set: 150


Fit your model on training data only. 

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

print("You should not do like that.") 
x_train = cv.fit_transform(x_train)      # --> Here I want to fit cv only with the train data. 
x_cv = cv.fit_transform(x_cv)
x_test = cv.fit_transform(x_test)        # --> By doing that you are causing data leakage problem. 
print(f'{x_train.shape = }, {x_cv.shape = }, {x_test.shape = }')
print("NOTE: THE NUMBER OF COLUMNS IN EACH OF THE VECTOR WONT BE SAME --> Which explains the problem.")
print('='*100)

You should not do like that.
x_train.shape = (476, 2145), x_cv.shape = (120, 752), x_test.shape = (150, 929)
NOTE: THE NUMBER OF COLUMNS IN EACH OF THE VECTOR WONT BE SAME --> Which explains the problem.


In [27]:
print("You should do like that.")
cv = CountVectorizer()
cv.fit(x_train) # fit has to happen only on train data

x_train = cv.transform(x_train)
x_cv = cv.transform(x_cv)
x_test = cv.transform(x_test)
print(f'{x_train.shape = }, {x_cv.shape = }, {x_test.shape = }')

You should do like that.
x_train.shape = (476, 2145), x_cv.shape = (120, 2145), x_test.shape = (150, 2145)


 YOU SHOULD NOT DO LIKE THIS 

1.  THE VOCABULARY SHOULD BUILT ONLY WITH THE WORDS OF TRAIN DATA
    vectorizer = CountVectorizer()
    x_train_bow = vectorizer.fit_transform(X_train)
    x_cv_bow = vectorizer.fit_transform(X_cv)
    x_test_bow = vectorizer.fit_transform(X_test)

2.  DATA LEAKAGE PROBLEM: IF WE DO LIKE THIS WE ARE LOOKING AT THE TEST DATA BEFORE MODELING
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transfomr(X)
    X_train, X_test, y_train, y_test = train_test_split(X_bow, Y, test_size=0.33)

3. YOU SHOULD PASS THE PROBABILITY SCORES NOT THE PREDICTED VALUES
    y_pred =  neigh.predict(X)
    roc_auc_score(y_ture,y_pred)


When you do: 
--> vec =  CountVectorizer()
Means it will initiate the CountVectorizer with default parameters.

--> vec.fit(Train_text): Means that internally it is only learning the vocabulary of `Text` i.e. unique n-grams

--> bag_of_words = vec.transform(Train_text) 
Means it is applying that learned parameters (vocabulary) to the data and thus giving you output i.e. words in Bag of words format. 
Now, as you should know that the vocabulary (unique n-grams) can be different for Train Text and Test Text thus they will give you different dimensional matrices for Train and Test. 

So what you should do is -  vec = CountVectorizer(), vec.fit(Train_text)
which learns the vocabulary of Train Text and then apply or transform your both Train Text and Test Text  using that  
learned vocabulary to ensure the same dimensions for both of them by doing -

bag_of_words_train = vec.transform(Train_text) and
bag_of_words_test = vec.transform(Test_text)

so to conclude
model =  CountVectorizer()
model.fit(train_text)
train_bow = model.transform(train_text)
test_bow = model.transform(test_text) 
or 
model =  CountVectorizer()
train_bow = model.fit_transform(train_text)
test_bow = model.transform(test_text) 